In [1]:
import sys
sys.path.append('../')
import exp
import regression as r

In [2]:
df = exp.get_wp_energy_data_8_day()
df.head()

,PET,VPD,air_temp,doy,precip,soil_temp,sw_in,wind_speed,year,wp_RNET,...,wp_bnd3,wp_bnd7,wp_evi,wp_lswi2,wp_ndvi,wp_LST.day,wp_LST.night,LW_IN.wp,LW_IN.si,LW_IN
0,7.33,0.808731,19.179167,195,0.0,22.320833,30.3156,4.958333,2012,20.798342,...,0.032962,0.136763,0.396594,0.353988,0.641934,31.567899,17.204530,29.105844,24.562860,29.105844
1,6.52,0.755945,19.325000,196,0.0,21.770833,29.6316,3.791667,2012,20.573593,...,0.032200,0.131800,0.407211,0.376832,0.654545,29.570000,17.390000,29.760736,26.096551,29.760736
2,6.92,0.858993,20.262500,197,0.0,21.908333,29.3472,4.137500,2012,20.475931,...,0.031425,0.128250,0.416158,0.394189,0.664269,31.097908,17.235624,30.299442,25.986418,30.299442
3,6.35,0.477617,16.791667,198,0.0,22.420833,28.8180,6.033333,2012,20.571045,...,0.030650,0.124700,0.425106,0.411547,0.673993,30.868718,17.248525,28.978868,25.122209,28.978868
4,5.13,0.556820,17.016667,199,0.0,21.529167,23.1732,4.350000,2012,16.757401,...,0.029875,0.121150,0.434053,0.428904,0.683716,30.657792,17.259663,30.653667,27.830367,30.653667


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1030 entries, 0 to 1029
Data columns (total 28 columns):
PET             1030 non-null float64
VPD             1030 non-null float64
air_temp        1030 non-null float64
doy             1030 non-null int64
precip          1030 non-null float64
soil_temp       1030 non-null float64
sw_in           1030 non-null float64
wind_speed      1030 non-null float64
year            1030 non-null int64
wp_RNET         1030 non-null float64
wp_ch4_gf       1030 non-null float64
wp_co2_gf       1030 non-null float64
wp_er           1030 non-null float64
wp_gpp          1030 non-null float64
wp_h            1030 non-null float64
wp_le           1030 non-null float64
wp_bnd1         1030 non-null float64
wp_bnd2         1030 non-null float64
wp_bnd3         1030 non-null float64
wp_bnd7         1030 non-null float64
wp_evi          1030 non-null float64
wp_lswi2        1030 non-null float64
wp_ndvi         1030 non-null float64
wp_LST.day      1030 no

In [4]:
train_cols = ["precip", "sw_in", "LW_IN.wp", "VPD", "air_temp", "PET", "wind_speed", "wp_LST.day",
             "wp_LST.night", "wp_evi", "wp_ndvi", "wp_lswi2"]
X, Y = exp.featurize(df, train_cols, ["wp_h"])
X, Y, scaler = r.preprocess(X, Y)
X.shape

(1030, 12)

In [5]:
r.random_forests_cross_val(X, Y, feature_names=train_cols)

Running Random Forests Cross Validation...
10-fold CV Acc Mean:  0.878024233427
CV Scores:  0.908545323761, 0.850217871536, 0.861791445599, 0.889979301352, 0.840157953888, 0.895691866239, 0.911817075362, 0.853366810606, 0.890007103312, 0.878667582611
OOB score: 0.888246466555
Feature Importances:
('sw_in', 0.21603996883114276)
('air_temp', 0.11459973383701712)
('PET', 0.10113497317028364)
('wp_LST.night', 0.10009351535214264)
('VPD', 0.092579523065221589)
('LW_IN.wp', 0.081517532179145494)
('wp_evi', 0.071804059521899569)
('wp_LST.day', 0.068573977786099818)
('wp_lswi2', 0.059656948504893635)
('wind_speed', 0.043109995389011925)
('wp_ndvi', 0.038491640217112447)
('precip', 0.012398132146029363)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=1, oob_score=True, random_state=None,
           verbose=0, warm_start=False)

In [6]:
r.xgb_trees_cross_val(X, Y, feature_names=train_cols)

Running Gradient Boosted Trees Cross Validation...
10-fold CV Acc Mean:  0.901587564637
CV Scores:  0.921298056366, 0.879097601244, 0.898304401101, 0.904635692039, 0.889092520934, 0.922278536548, 0.89946258833, 0.880470593965, 0.919601707305, 0.901633948537
Feature Importances:
('sw_in', 0.15609290226651976)
('VPD', 0.11664424461929902)
('wp_LST.night', 0.11126672749246624)
('LW_IN.wp', 0.094875592359430633)
('PET', 0.091054573408512293)
('wp_LST.day', 0.088546337665651148)
('wp_evi', 0.073798880100752126)
('wp_lswi2', 0.07073728381717509)
('wind_speed', 0.065958654126702682)
('air_temp', 0.059705005326297515)
('wp_ndvi', 0.059619418990926799)
('precip', 0.011700379826266833)


GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=3, max_features='sqrt', max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [7]:
r.svc_cross_val(X, Y)

Running SVC Cross Validation...
10-fold CV Acc Mean:  0.878055211267
CV Scores:  0.900599178812, 0.893500791351, 0.889543012496, 0.891165858629, 0.83708908948, 0.907843457664, 0.896388790866, 0.831090630569, 0.888723435487, 0.844607867314


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [8]:
r.dnn_cross_val(X, Y)

Running Neural Network Cross Validation...
Step #1, avg. loss: 10.89695
Step #501, epoch #50, avg. loss: 0.95996
Step #1001, epoch #100, avg. loss: 0.39009
Step #1501, epoch #150, avg. loss: 0.28950
Step #2001, epoch #200, avg. loss: 0.23156
Step #2501, epoch #250, avg. loss: 0.19498
Step #3001, epoch #300, avg. loss: 0.16769
Step #3501, epoch #350, avg. loss: 0.14978
Step #4001, epoch #400, avg. loss: 0.13580
Step #4501, epoch #450, avg. loss: 0.11981
Step #1, avg. loss: 9.27540
Step #501, epoch #50, avg. loss: 0.98140
Step #1001, epoch #100, avg. loss: 0.38715
Step #1501, epoch #150, avg. loss: 0.26860
Step #2001, epoch #200, avg. loss: 0.21513
Step #2501, epoch #250, avg. loss: 0.18097
Step #3001, epoch #300, avg. loss: 0.15673
Step #3501, epoch #350, avg. loss: 0.13758
Step #4001, epoch #400, avg. loss: 0.13064
Step #4501, epoch #450, avg. loss: 0.12018
Step #1, avg. loss: 9.93125
Step #501, epoch #50, avg. loss: 0.94472
Step #1001, epoch #100, avg. loss: 0.38476
Step #1501, epoch 

TensorFlowEstimator(batch_size=100, class_weight=None,
          continue_training=False, early_stopping_rounds=None,
          keep_checkpoint_every_n_hours=10000, learning_rate=0.1,
          max_to_keep=5, model_fn=<function tanh_dnn at 0x10f130488>,
          n_classes=0, num_cores=4, optimizer='SGD', steps=5000,
          tf_master='', tf_random_seed=42, verbose=1)